In [3]:
%pip install nltk
%pip install beautifulsoup4
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [19]:
import nltk
import string
import os
import collections

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

from bs4 import BeautifulSoup
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to /home/raul/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/raul/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/raul/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [20]:
# Read all html files in path and transform in text
#   If you get an UnicodeDecodeError remove file content in 'file' varaible
path = './html'
translate_table = dict((ord(char), ' ') for char in string.punctuation)
token_dict = {}
for subdir, dirs, files in os.walk(path):
    for file in files:
        file_path = subdir + os.path.sep + file
        shakes = open(file_path, 'r')
        try:
          html = shakes.read()
        except Exception:
          continue
        # Extract text from html
        text = BeautifulSoup(html).get_text().encode('ascii', 'ignore')
        # Lowercase and remove punctuation
        lowers = str(text.lower())
        no_punctuation = lowers.translate(translate_table)
        token_dict[file] = no_punctuation

# Show number of reading files
len(token_dict)

111

In [24]:
# Tokenizer function
def process_text(text, stem=True):
    """Tokenize text and stem words removing punctuation"""
    text = text.translate(string.punctuation)
    tokens = word_tokenize(text)
    if stem:
        stemmer = PorterStemmer()
        tokens = [stemmer.stem(t) for t in tokens]
    return tokens

# Transform texts to Tf-Idf coordinates
# (http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)
#  If you get an "UserWarning: Your stop_words may be inconsistent" ignore it
stop_words = [process_text(w)[0] for w in stopwords.words('english')]
vectorizer = TfidfVectorizer(tokenizer=process_text,
                             stop_words=stop_words,
                             max_df=0.5,
                             min_df=0.1,
                             lowercase=True)
tfidf_model = vectorizer.fit_transform(token_dict.values())

# Show dimensions of tfidf_model
tfidf_model

/home/raul/Escritorio/busquedas_tid/.venv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/raul/Escritorio/busquedas_tid/.venv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['becau'] not in stop_words.
  warnings.warn(


<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 11325 stored elements and shape (111, 561)>

In [25]:
# Show selected tokens
#   On old versions of scikit-learn change 'get_feature_names_out' method by
#   'get_feature_names'
feature_names = vectorizer.get_feature_names_out()
feature_names

array(['0', '000', '1', '10', '100', '11', '12', '13', '14', '15', '16',
       '17', '18', '19', '2', '20', '2023', '2024', '2026', '21', '22',
       '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32',
       '35', '36', '4', '40', '49', '5', '50', '500', '6', '7', '8', '9',
       'access', 'account', 'achiev', 'across', 'action', 'activ', 'ad',
       'addit', 'address', 'advanc', 'age', 'agre', 'allow', 'also',
       'alway', 'america', 'american', 'angel', 'announc', 'annual',
       'app', 'appli', 'applic', 'april', 'around', 'articl', 'assist',
       'athlet', 'avail', 'award', 'back', 'base', 'basketbal', 'becom',
       'begin', 'benefit', 'best', 'better', 'bodi', 'book', 'box',
       'break', 'bring', 'build', 'busi', 'c', 'call', 'camp', 'canada',
       'care', 'career', 'celebr', 'center', 'central', 'challeng',
       'champion', 'championship', 'chang', 'check', 'choic', 'choos',
       'citi', 'class', 'click', 'close', 'club', 'coach', 'code',
      

In [26]:
# Cluster texts using K-Means
# (http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html)
#   On old versions of scikit-learn remove 'n_init' parameter
km_model = KMeans(n_clusters=3, n_init='auto', verbose=0)
km_model.fit(tfidf_model)

# Print clusters
clusters = collections.defaultdict(list)
for idx, label in enumerate(km_model.labels_):
    clusters[label].append(idx)

dict(clusters)

# Print number of elements of each cluster
for key, elements in dict(clusters).items():
    print(str(key) + ':', len(elements))

# Print labels of each url webpage
key = list(token_dict.keys())
for idx, label in enumerate(km_model.labels_):
    print(str(label) + ':', key[idx].replace('_','/').replace('.html',''))

# Print 4 most relevant tokens of each cluster
kmcc = km_model.cluster_centers_.copy()
for idx, item in enumerate(dict(clusters).items()):
    print(str(item[0]) + ':')
    for j in range(4):
        idxmax = kmcc[idx].argmax()
        print('  ', feature_names[idxmax], ' - ', kmcc[idx][idxmax])
        kmcc[idx][idxmax] = 0.0
    print()

2: 30
0: 69
1: 12
2: https://www.usatriathlon.org/
0: https://www.mbusa.com/en/home
2: https://www.usaartisticswim.org/
1: https://es.wikipedia.org/wiki/C%2B%2B
0: https://cplusplus.com/
0: https://www.open-std.org/jtc1/sc22/wg21/
2: http://www.ussoccer.com/
0: https://la28.org/
1: https://computerhoy.20minutos.es/tecnologia/bjarne-stroustrup-aprender-programar-1445288
0: https://2026specialolympicsusagames.org/
0: https://educationusa.state.gov/
2: https://www.usab.com/
0: https://wiki.sei.cmu.edu/confluence/pages/viewpage.action?pageId=88046682
0: https://www.usaswimming.org/
0: https://www.codecademy.com/learn/learn-c-plus-plus
0: https://www.w3schools.com/CPP/default.asp
1: https://visualstudio.microsoft.com/es/vs/features/cplusplus/
0: https://www.specialolympics.org/
0: https://www.usa829.org/
2: https://www.udacity.com/course/c-plus-plus-nanodegree--nd213
2: https://www.unicefusa.org/
0: https://en.wikipedia.org/wiki/Summer/Olympic/Games
2: https://usrowing.org/teams/olympic
1: 